In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from ipywidgets import FloatProgress
from IPython.display import display
from osgeo import gdal, ogr

%matplotlib inline

In [2]:
os.listdir(os.path.join('..','MyFiles','Education','Mosey Project Docs'))

['London map trial.ipynb',
 'Map Go.ipynb',
 'London map trial-Copy1.ipynb',
 'merged.csv',
 'photos_nearby.csv',
 'Folium.ipynb',
 'gmaps_nearby.csv',
 'Documenting Code.ipynb',
 'Instagram+Google_Modified.ipynb',
 '2017-01-08-city-of-london-street_ScoreTesting.csv',
 'Simple Routing App-Copy1.ipynb',
 'Untitled.ipynb',
 'test_multiprocessing.ipynb',
 'working_routing_code.ipynb',
 'map_network.ipynb',
 'experimenting.ipynb',
 'Cityof London+Metropolitan_Aug-Sept2017_Score.csv',
 'Simple Routing App-Copy2.ipynb',
 'poi.csv',
 'POI-Holidays.csv',
 'tl_2013_06_prisecroads.shx',
 'final_info.csv',
 'Threading.ipynb',
 'networkx_test.py',
 'tl_2013_06_prisecroads.shp',
 'pathpointstb2.csv',
 'dinda-map-test.ipynb',
 'gis_osm_roads_free_1.dbf',
 'Simple Routing App.ipynb',
 'combined_routingcode_gmaps.ipynb',
 'test_multiprocessing-Copy1.ipynb',
 'Untitled-checkpoint.ipynb',
 '.ipynb_checkpoints',
 'gis_osm_roads_free_1.shx',
 'gis_osm_roads_free_1.shp',
 'merged1.csv',
 'point_of_interest

In [3]:
# Get POI/Crime Data

In [4]:
crime_df = pd.read_csv(os.path.join('..','MyFiles','Education','Mosey Project Docs','Cityof London+Metropolitan_Aug-Sept2017_Score.csv'))
crime_df2 = pd.read_csv(os.path.join('..','MyFiles','Education','Mosey Project Docs','2017-01-08-city-of-london-street_ScoreTesting.csv'))
print('Done')
print(crime_df.shape)
print(crime_df2.shape)

Done
(136311, 17)
(3425, 17)


In [5]:
poi_df = pd.read_csv(os.path.join('..','MyFiles','Education','Mosey Project Docs','POI-Holidays.csv'))

In [6]:
g = nx.read_shp(os.path.join('..','MyFiles','Education','Mosey Project Docs',"gis_osm_roads_free_1.shp"))


In [7]:
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [8]:
sgs = list(connected_component_subgraphs(
                   g.to_undirected()))
print('connected g')

connected g


In [9]:
largest = np.argmax([len(sg) 
                    for sg in sgs])
print('found largest sg')
sg = sgs[largest]

found largest sg


In [10]:
for v in sg.edges():
    
    d = {'longitude':(v[0][0]),'latitude':v[0][1]}
d

{'longitude': -0.3499169, 'latitude': 51.5751178}

In [11]:
#pos0 = (51.507554,-0.078958)
#Southbank Center
pos0 = (51.507028,-0.116392)
#Angel Station
pos1 = (51.532938,-0.105841)
#Exmouth
#pos1 = (51.525372,-0.110049)

In [12]:
def get_path(n0, n1):
    """If n0 and n1 are connected nodes in the 
    graph, this function returns an array of point
    coordinates along the road linking these two nodes."""
    return np.array(json.loads(
                sg[n0][n1]['Json'])['coordinates'])

In [13]:
EARTH_R = 6372.8
def geocalc(lat0, lon0, lat1, lon1):
    """Return the distance (in km) between two
    points in geographical coordinates."""
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt(
            (np.cos(lat1)*np.sin(dlon))**2
            +(np.cos(lat0)*np.sin(lat1) 
            -np.sin(lat0)*np.cos(lat1)* \
            np.cos(dlon))**2)
    x = np.sin(lat0)*np.sin(lat1) + \
        np.cos(lat0)*np.cos(lat1)*np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c

In [14]:
def get_path_length(path):
    return np.sum(geocalc(
                    path[1:,0], path[1:,1],
                    path[:-1,0], path[:-1,1]))#change these two 
# to iterate over all crime incidents w/in X radius of edge first
#two points.
def get_crime_path_length(path,crime):
    return np.sum(geocalc(
                    path[1:,0], path[1:,1],
                    path[:-1,0], path[:-1,1]))#change these two 

In [15]:
for n0, n1 in sg.edges_iter():
    path = get_path(n0, n1)
    distance = get_path_length(path)
    #print(distance)
    sg.edge[n0][n1]['distance'] = distance
#print(sg)
print('Step 1 Done')

Step 1 Done


In [18]:
f = FloatProgress(min=0, max=len(sg))
display(f)
for n0,n1 in sg.edges_iter():
    f.value+=1
    cdistance=0
    path = get_path(n0,n1)
    for i in range(round(len(crime_df)/10)):
        cdistance+= geocalc(path[0,0],path[0,1],
                            crime_df.at[i,'Latitude'],
                            crime_df.at[i,'Longitude'])
        +geocalc(path[:-1,0],path[:-1,1],
                            crime_df.at[i,'Latitude'],
                            crime_df.at[i,'Longitude'])
        +float(crime_df.at[i,'Final Score 2'])
    sg.edge[n0][n1]['Crime_Weight']=cdistance
                                      
print('Step 2 done')

FloatProgress(value=0.0, max=60904.0)

KeyboardInterrupt: 

In [ ]:
for n0,n1 in sg.edges_iter():
    cdistance=0
    #print(n0)
    #print(n1)
    path = get_path(n0,n1)
    #print(path)
    for i in range(len(poi_df)):
        cdistance+= geocalc(path[:-1,0],path[:-1,1],
                            poi_df.at[i,'latitude'],
                            poi_df.at[i,'longitude'])
        #+ geocalc(path[1,0],path[1,-1],
        #         poi_df.at[i,'latitude'],
        #                    poi_df.at[i,'longitude'])
    sg.edge[n0][n1]['POI_Weight']=np.sum(cdistance)
                                      
print('Step 3 done')

In [ ]:
nodes = np.array(sg.nodes())
#print(len(nodes))
#print(nodes)
# Get the closest nodes in the graph.
#print(nodes[:,::-1])
pos0_i = np.argmin(np.sum(
                    (nodes[:,::-1] - pos0)**2,
                     axis=1))
#print("pos0_i="+str(pos0_i))
#print(pos0)
#print(nodes[pos0_i])
pos1_i = np.argmin(np.sum(
                        (nodes[:,::-1] - pos1)**2,
                         axis=1))
#print(pos1_i)
#print(pos1)
#print(nodes[pos1_i])

In [ ]:
path = nx.shortest_path(sg, 
            source=tuple(nodes[pos0_i]), 
            target=tuple(nodes[pos1_i]),
            weight='Crime_Weight')
len(path)
print(type(path))
print(path)

In [ ]:
path2 = nx.shortest_path(sg, 
            source=tuple(nodes[pos0_i]), 
            target=tuple(nodes[pos1_i]),
            weight='POI_Weight')
len(path2)
print(type(path2))
print(path2)

In [ ]:
i = 1
endlist = []
for tup in path2:
    pair = list(tup)
    pair.insert(0,i)
    i+=1
    endlist.append(pair)
print(endlist)
df = pd.DataFrame(endlist, columns=["path","longitude", "latitude"] )
df.to_csv('pathpointstb2.csv', index=False)